In [2]:
# 영상을 받아들일 카메라 인스턴스 생성 

from jetbot import Camera 
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=720, height=720)  # 720x720 픽셀 이미지 사용

In [3]:
# 서보모터의 초기화 위치 설정
# sudo chmod 666 /dev/ttyTHS1 해당 명령어로 원한 수정 해주어야함


from servoserial import ServoSerial
import time

pantilt = ServoSerial()  # Pantilt 서보 인스턴스 생성

# 초기 카메라 방향을 좌우의 중간, 상하의 약간 위를 바라보도록 설정
pantilt.Servo_serial_double_control(1,2048,2,2048+200)
time.sleep(0.5)

serial Open!
83
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\xc8\x00\nS'


In [4]:
# 검출 영역의 좌표와 크기
# 검출 대상인 얼굴이 검출 된 좌표의 크기와 영역, 나중에 박스로 표시함
global detect_x, detect_y, detect_w, detect_h

detect_x = detect_y = detect_w = detect_h = 0

In [4]:
# 키메라 프리뷰 위젯 만들기

import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

detect_img = widgets.Image(format='jpeg', width=300, height=300)  # 프리뷰 화면은 300x300 픽셀로 설정
display(detect_img)

Image(value=b'', format='jpeg', height='300', width='300')

In [5]:
# HARR Ccascade Classifier 학습 데이터 로딩
#아래 셋 중 하나 골라서 사용

import cv2

detect_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')  # 정면 얼굴 검출용, 해당 파일을 같은 다렉토리에 둔 후 작업한다, 아니면 쳥로를 찾아서 정확하게 입력한다
# https://diy-project.tistory.com/79#google_vignette
# detect_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')  # 몸 전체 검출용
# detect_cascade = cv2.CascadeClassifier('haarcascade_upperbody.xml')  # 상반신 검출용

In [6]:
# 카메라 영상에서 검출된 영역을 표시하는 메인 루프

while True:
    frame = camera.value  # 카메라에서 영산 프레임 이미지를 가져옴 
    frame = cv2.resize(frame, (300,300))  # 이미지를 학습 테이터와 동일한 크기로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백 이미지로 변환하여 검출에 사용
    
    bodies = detect_cascade.detectMultiScale(gray)  # 이미지 내에서 신체 검출, 흑백으로 변환된 이미지에서 크기가 다른 객체를 검출하는 함수    
    
    if len(bodies) > 0:
        (detect_x, detect_y, detect_w, detect_h) = bodies[0]  # 좌표와 크기
        
        # 검출 영역을 사각 박스로 표시
        # (이미지, 시작좌표, 종료좌표, BGR형태의 컬러 , 선의 두깨(픽셀 단위))
        cv2.rectangle(frame, (detect_x, detect_y), (detect_x+detect_h, detect_y+detect_w), (0,255,0),4)
        
    # 결과 이미지를 프리뷰 위젯에 표시
    detect_img.value=bgr8_to_jpeg(frame)  # 사각 박스로 표시한 이미지를 표시

KeyboardInterrupt: 